### Notebook to merge machine read triples after extraction and processing from INDRA/REACH and SemRep

In [1]:
import pandas as pd
import numpy as np
import pickle
from rdflib.namespace import OWL, RDF, RDFS
import os
import pickle, json

#Create networkx graph from triples
import glob
import hashlib
import json
import networkx as nx  # type: ignore
import os
import os.path

from collections import Counter  # type: ignore
from more_itertools import unique_everseen  # type: ignore
from rdflib import BNode, Graph, Literal, Namespace, URIRef  # type: ignore
from rdflib.namespace import OWL, RDF, RDFS  # type: ignore
from rdflib.plugins.serializers.nt import _quoteLiteral  # type: ignore
import subprocess

from tqdm import tqdm  # type: ignore
from typing import Dict, List, Optional, Set, Tuple, Union

In [ ]:
##Previous run v1.0.1 - 2022-06-16
##Latest run v2.0.0 - 2023-04-23

In [4]:
##read in machine reading graphs
g1 = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machineread_semrep_version1.nt'
g2 = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machineread_reach_version1.nt'

In [2]:
#if merging ntriples - concatenate files
#if merging gpickle - use nx.compose function. Also merge nodelabels and fix labels

In [5]:
with open(g1,'r') as file1:
    g = file1.read()
graph1 = g.split('\n')
type(graph1)

list

In [6]:
graph1[0]

'<http://purl.obolibrary.org/obo/CHEBI_28748> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/CHEBI_9948> .'

In [7]:
with open(g2,'r') as file2:
    g = file2.read()
graph2 = g.split('\n')
graph2[0]

'<http://purl.obolibrary.org/obo/CHEBI_16057> <http://www.w3.org/2000/01/rdf-schema#label> "Dimethylphenylpiperazinium_Iodide" .'

In [8]:
print(len(graph1), len(graph2))

38141 47287


In [9]:
output_file = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machine_read_merged_20230423.nt'

In [10]:
with open(output_file, 'w') as fileo:
    for gt1 in graph1:
        fileo.write(gt1)
        fileo.write('\n')
    for gt2 in graph2:
        fileo.write(gt2)
        fileo.write('\n')
file1.close()
file2.close()
fileo.close()

In [11]:
with open(output_file,'r') as filei:
    g = filei.read()
mr = g.split('\n')
len(mr)

85429

In [13]:
inferred_file = '/home/sanya/PheKnowLatorv3/literature_graphs/closure_output/inferred-transitive-and-symmetric.ntriples'
with open(inferred_file,'r') as file2:
    g2 = file2.read()
mrinf = g2.split('\n')
len(mrinf)

21633

In [14]:
mr[0]

'<http://purl.obolibrary.org/obo/CHEBI_28748> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/CHEBI_9948> .'

In [15]:
mrinf_new = []
for item in mrinf:
    if mrinf != '':
        temp = item.replace('>', '> ')
        mrinf_new.append(temp)
len(mrinf_new)

21633

In [16]:
outmerged = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machine_read_merged_with_closure_20230423.nt'
with open(outmerged, 'w') as fileo:
    for gt1 in mr:
        if gt1 != '':
            fileo.write(gt1)
            fileo.write('\n')
    for gt2 in mrinf_new:
        if gt2 != '':
            fileo.write(gt2)
            fileo.write('\n')
fileo.close()
filei.close()
file2.close()

In [ ]:
##all graphs merged -

In [17]:
outmerged = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machine_read_merged_with_closure_20230423.nt'
with open(outmerged,'r') as filei:
    g = filei.read()
mr = g.split('\n')
len(mr)

107057

In [21]:
mr[-2]

'<http://purl.obolibrary.org/obo/CHEBI_47519> <http://purl.obolibrary.org/ob'

In [23]:
outfile = '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machine_read_merged_with_closure_20230423.nt'
with open(outfile,'w') as fileo:
    for i in range(0, len(mr)-2):
        fileo.write(mr[i])
        fileo.write('\n')
fileo.close()

In [25]:
mr[-2]

'<http://purl.obolibrary.org/obo/CHEBI_47519> <http://purl.obolibrary.org/ob'

#### Merge gpickle files

In [1]:
import networkx as nx
g1 = nx.read_gpickle('/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machineread_semrep_version1.gpickle')
g2 = nx.read_gpickle('/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machineread_reach_version1.gpickle')
ginf = nx.read_gpickle('/home/sanya/PheKnowLatorv3/literature_graphs/closure_output/machineread_inferred_symmetric_transitive.gpickle')

In [2]:
print(len(g1), len(g2), len(ginf))

4288 6217 2174


In [3]:
def get_stats(g):
    nodes = nx.number_of_nodes(g)
    edges = nx.number_of_edges(g)
    density = nx.density(g)
    avg_deg = float(edges)/nodes
    print(nodes, edges, density, avg_deg)

In [4]:
##compose graphs
g3 = nx.compose(g1,g2)
g_all = nx.compose(g3,ginf)

In [33]:
print('SemRep graph')
get_stats(g1)
print('Reach graph')
get_stats(g2)
print('Inferred graph')
get_stats(ginf)
print('Combined graphs')
get_stats(g_all)

SemRep graph
4288 33594 0.0018274834713764975 7.834421641791045
Reach graph
6217 40245 0.0010414059593728245 6.473379443461477
Inferred graph
2174 18450 0.003905504157192203 8.486660533578657
Combined graphs
8782 84569 0.0010966645002845241 9.629810976998407


In [5]:
##save graph
nx.write_gpickle(g_all, '/home/sanya/PheKnowLatorv3/literature_graphs/output_graphs/machine_read_merged_20230423.gpickle')

### Merge nodelabels for reach and semrep

In [2]:
import json
with open('/home/sanya/ontologies/ro.json', 'r') as file:
    ro = json.load(file)
len(ro)

1

In [17]:
relation_labels = {}
for node in ro['graphs'][0]['nodes']:
    try:
        if 'lbl' in node.keys():
            relation_labels[node['id']] = node['lbl']
    except:
        print(node)
        break
len(relation_labels)

767

In [21]:
relation_labels['http://purl.obolibrary.org/obo/RO_0002434']

'interacts with'

In [22]:
import pickle

In [23]:
nodelabels_sem = 'output_graphs/machineread_semrep_NodeLabels.pickle'
nodelabels_r = 'output_graphs/reach_version1_NodeLabels.pickle'

In [24]:
with open(nodelabels_sem, 'rb') as filep1:
    sem_labels = pickle.load(filep1)
len(sem_labels)

4304

In [25]:
with open(nodelabels_r, 'rb') as filep2:
    r_labels = pickle.load(filep2)
len(r_labels)

6230

In [26]:
##combine labels
all_labels = {**sem_labels, **r_labels}
len(all_labels)

8758

In [27]:
##fix labels of relations using relation_labels
for key in all_labels:
    if key in relation_labels.keys():
        all_labels[key]['label'] = relation_labels[key]
len(all_labels)

8758

In [30]:
all_labels['http://purl.obolibrary.org/obo/RO_0002448']

{'entity_type': 'RELATIONS', 'label': 'directly regulates activity of'}

In [31]:
##save labels
with open('output_graphs/machine_read_merged_20230423_NodeLabels.pickle', 'wb') as filep:
    pickle.dump(all_labels, filep)

## Merge ntriples files for PheKnowLator and machine read

Not run in 2023 update yet

In [2]:
pl_file = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS.nt'
mr_file = '/home/sanya/PheKnowLatorv2/machine_read/output_graphs/machine_read_merged_with_closure_20220616.nt'
outfile_merged = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/merged_graphs/PheKnowLator_machine_read_merged_instance_based_OWLNETS_no_labels_20220616.nt'

In [3]:
with open(pl_file,'r') as filep:
    g = filep.read()
pl = g.split('\n')

In [4]:
len(pl)

21691793

In [5]:
with open(mr_file,'r') as filem:
    g = filem.read()
mr = g.split('\n')

In [6]:
len(mr)

33844

In [11]:
mr[-2]

'<http://purl.obolibrary.org/obo/CHEBI_48705> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/PR_000015800> .'

In [7]:
##not writing labels to file
with open(outfile_merged, 'w') as fileout:
    newline = ''
    for i in range(0, len(pl)-1):
        if pl[i] == '':
            continue
        else:
            fileout.write(newline+pl[i])
            newline = '\n'
    newline = '\n'
    for i in range(0, len(mr)-1):
        if 'label' in mr[i] or mr[i] == '':
            continue
        else:
            fileout.write(newline+mr[i])
            newline = '\n'
fileout.close()

In [ ]:
#merged == 21725642 ()
#no labels = 21720940

In [8]:
##check file
with open(outfile_merged, 'r') as filein:
    g = filein.read()
merged = g.split('\n')
len(merged)

21720940

In [15]:
merged[-1]

'<http://purl.obolibrary.org/obo/CHEBI_48705> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/PR_000015800> .'

In [16]:
merged[0]

'<http://purl.obolibrary.org/obo/CLO_0005160> <http://purl.obolibrary.org/obo/BFO_0000050> <http://purl.obolibrary.org/obo/NCBITaxon_9606> .'

In [18]:
##remove last line from PL graph
pl_file_new = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_20220616.nt'

In [20]:
with open(pl_file_new, 'w') as filepl:
    newline = ''
    for item in pl:
        if item != '':
            filepl.write(newline+item)
            newline = '\n'
filepl.close()

### Get details on metanodes and metaedges in MR graphs (with inferred and without)

In [ ]:
mr_file = '/home/sanya/PheKnowLatorv2/machine_read/output_graphs/machine_read_merged_with_closure_20220616.nt'

In [3]:
with open(file1,'r') as filei:
    g = filei.read()
mr = g.split('\n')
len(mr)

19361

In [46]:
with open(file2,'r') as fileo:
    g = fileo.read()
mr_inf = g.split('\n')
len(mr_inf)

33851

In [4]:
with open(mr_file,'r') as filem:
    g = filem.read()
mr = g.split('\n')
len(mr)

33844

In [5]:
mr[0]

'<http://purl.obolibrary.org/obo/CHEBI_22586> <http://purl.obolibrary.org/obo/RO_0002434> <http://purl.obolibrary.org/obo/CHEBI_16261> .'

In [6]:
#find all unique metanodes and metaedges

subjects = {}
objects = {}
rels = {}
for triple in mr:
    if triple == '':
        continue
    triple_list = triple.split(' ')
    subj = triple_list[0].replace('>', '')
    obj = triple_list[2].replace('>', '')
    pred = triple_list[1].replace('>','')
    
    subj_id = subj.split('/')[-1].split('_')[0]
    if subj_id not in subjects:
        subjects[subj_id] = 1
    else:
        subjects[subj_id] += 1
    pred_id = pred.split('/')[-1]
    if pred_id not in rels:
        rels[pred_id] = 1
    else:
        rels[pred_id] += 1
    
    if triple_list[1] == '<http://www.w3.org/2000/01/rdf-schema#label>':
        obj_id = 'Literal_Object'
    else:
        obj_id = obj.split('/')[-1].split('_')[0]
    
    if obj_id not in objects:
        objects[obj_id] = 1
    else:
        objects[obj_id] += 1


In [7]:
subjects

{'CHEBI': 24029,
 'UBERON': 872,
 'DOID': 11,
 'PR': 3668,
 'napdi': 1735,
 'MONDO': 371,
 'NCBITaxon': 468,
 'CL': 699,
 'GO': 1067,
 'SO': 454,
 'CLO': 66,
 'cellline#human': 50,
 'HP': 278,
 'PW': 8,
 'PATO': 59,
 'OBO': 4,
 'SLC47A1[SYNONYM': 1,
 'SLC47A1SYNONYM': 3}

In [8]:
objects

{'CHEBI': 11023,
 'Literal_Object': 4695,
 'NCBITaxon': 1736,
 'PR': 10774,
 'napdi': 722,
 'MONDO': 689,
 'UBERON': 1095,
 'GO': 2071,
 'CL': 350,
 'CLO': 53,
 'PW': 19,
 'SO': 219,
 'HP': 357,
 'DOID': 27,
 'OBO': 2,
 'PATO': 3,
 'SLC47A1[SYNONYM': 6,
 'cellline#human': 1,
 'SLC47A1SYNONYM': 1}

In [9]:
rels

{'RO_0002434': 4796,
 'rdf-schema#label': 4695,
 'BFO_0000050': 2243,
 'RO_0002606': 505,
 'RO_0002213': 11971,
 'RO_0002449': 3458,
 'RO_0002599': 53,
 'RO_0002598': 236,
 'RO_0002490': 1434,
 'RO_0002610': 110,
 'RO_0002596': 353,
 'RO_0002212': 153,
 'RO_0002566': 126,
 'RO_0003000': 248,
 'RO_0003302': 27,
 'BFO_0000063': 1,
 'RO_0003309': 1,
 'RO_0002448': 2518,
 'RO_0011010': 354,
 'RO_0011009': 438,
 'RO_0002447': 64,
 'GO_0006470': 25,
 'GO_0006479': 7,
 'GO_0006486': 6,
 'GO_0006473': 3,
 'GO_0018126': 8,
 'GO_0006482': 3,
 'RO_0002436': 5,
 'RO_0002480': 1,
 'GO_0016579': 1}

In [19]:
#get number of each metaedge per subject and object metanode - at node level 
##don't include labels
metaedge_nodeonly = {}
for triple in mr:
    if triple == '':
        continue
    triple_list = triple.split(' ')
    subj = triple_list[0].replace('>', '')
    obj = triple_list[2].replace('>', '')
    
    subj_id = subj.split('/')[-1].split('_')[0]
    if triple_list[1] == '<http://www.w3.org/2000/01/rdf-schema#label>':
        obj_id = ''
    else:
        obj_id = obj.split('/')[-1].split('_')[0]
    
    if obj_id != '':
        edgeinfo = subj_id + '_'+ obj_id
    
    if edgeinfo not in metaedge_nodeonly:
        metaedge_nodeonly[edgeinfo] = 1
    else:
        metaedge_nodeonly[edgeinfo] += 1
metaedge_nodeonly

{'CHEBI_CHEBI': 5968,
 'UBERON_NCBITaxon': 211,
 'CHEBI_PR': 3664,
 'CHEBI_NCBITaxon': 548,
 'CHEBI_napdi': 393,
 'CHEBI_MONDO': 509,
 'CHEBI_UBERON': 324,
 'napdi_CHEBI': 627,
 'MONDO_MONDO': 47,
 'napdi_MONDO': 73,
 'NCBITaxon_MONDO': 63,
 'CL_NCBITaxon': 72,
 'CHEBI_GO': 1408,
 'NCBITaxon_NCBITaxon': 63,
 'napdi_GO': 88,
 'GO_UBERON': 26,
 'GO_PR': 131,
 'napdi_PR': 557,
 'PR_NCBITaxon': 54,
 'PR_CHEBI': 678,
 'GO_CL': 19,
 'CL_UBERON': 100,
 'UBERON_UBERON': 148,
 'GO_CHEBI': 164,
 'napdi_CLO': 2,
 'CL_PR': 114,
 'UBERON_CHEBI': 36,
 'SO_CHEBI': 51,
 'CL_CL': 18,
 'SO_PR': 96,
 'PR_napdi': 48,
 'GO_NCBITaxon': 25,
 'PR_UBERON': 53,
 'SO_napdi': 6,
 'CHEBI_CL': 149,
 'CLO_UBERON': 2,
 'CHEBI_PW': 7,
 'CL_MONDO': 2,
 'GO_napdi': 19,
 'UBERON_CL': 1,
 'PR_MONDO': 50,
 'PR_PR': 634,
 'PR_SO': 16,
 'cellline#human_PR': 6,
 'MONDO_HP': 13,
 'CHEBI_SO': 58,
 'SO_SO': 6,
 'CHEBI_CLO': 35,
 'cellline#human_UBERON': 6,
 'PR_CL': 70,
 'CHEBI_DOID': 20,
 'napdi_OBO': 1,
 'CLO_PR': 41,
 'MONDO_

In [ ]:
#get number of each metaedge per subject and object metanode
##don't include labels
metaedge = {}
for triple in mr:
    if triple == '':
        continue
    triple_list = triple.split(' ')
    subj = triple_list[0].replace('>', '')
    obj = triple_list[2].replace('>', '')
    pred = triple_list[1].replace('>','')
    
    subj_id = subj.split('/')[-1].split('_')[0]
    pred_id = pred.split('/')[-1]
    if triple_list[1] == '<http://www.w3.org/2000/01/rdf-schema#label>':
        obj_id = ''
    else:
        obj_id = obj.split('/')[-1].split('_')[0]
    
    if obj_id != '':
        edgeinfo = subj_id + '-'+ pred_id + '-' + obj_id
    
    if edgeinfo not in metaedge:
        metaedge[edgeinfo] = 1
    else:
        metaedge[edgeinfo] += 1
metaedge

In [21]:
for key in metaedge:
    if metaedge[key] >= 100:
        print(key, metaedge[key])

CHEBI-RO_0002434-CHEBI 1393
UBERON-BFO_0000050-NCBITaxon 207
CHEBI-RO_0002434-PR 585
CHEBI-RO_0002606-NCBITaxon 380
CHEBI-RO_0002213-CHEBI 592
CHEBI-RO_0002490-PR 246
CHEBI-RO_0002449-PR 1147
CHEBI-RO_0002490-CHEBI 1176
CHEBI-RO_0002449-CHEBI 1487
napdi-RO_0002434-CHEBI 165
CHEBI-RO_0002213-PR 628
CHEBI-RO_0002606-MONDO 144
napdi-RO_0002449-PR 178
napdi-RO_0002213-PR 123
UBERON-BFO_0000050-UBERON 146
napdi-RO_0002449-CHEBI 216
CHEBI-RO_0002449-GO 497
napdi-RO_0002434-PR 124
PR-RO_0002449-CHEBI 175
CHEBI-RO_0002596-UBERON 116
PR-RO_0002449-PR 152
CHEBI-RO_0002434-napdi 170
PR-RO_0002449-GO 137
CHEBI-RO_0002448-PR 478
CHEBI-RO_0011010-CHEBI 118
CHEBI-RO_0011009-PR 280
CHEBI-RO_0002448-CHEBI 849
CHEBI-RO_0002448-GO 535
CHEBI-RO_0011010-PR 187
CHEBI-RO_0002448-HP 160
PR-RO_0002448-GO 250
PR-RO_0002448-PR 141
PR-RO_0002448-CHEBI 264
CHEBI-RO_0011009-CHEBI 157
CHEBI-RO_0002449-HP 130


In [22]:
##edges in inferred
inferred_file = '/home/sanya/PheKnowLatorv2/machine_read/closure_output/inferred-transitive-and-symmetric.ntriples'
with open(inferred_file,'r') as file2:
    g2 = file2.read()
mrinf = g2.split('\n')
len(mrinf)

14491

In [39]:
mrinf[0].split('>').

['<http://purl.obolibrary.org/obo/NCBITaxon_1224',
 '<http://purl.obolibrary.org/obo/RO_0002434',
 '<http://purl.obolibrary.org/obo/NCBITaxon_2',
 '.']

In [48]:
mr_inf[]

'<http://purl.obolibrary.org/obo/CHEBI_22586> <http://purl.obolibrary.org/obo/RO_0002434> <http://purl.obolibrary.org/obo/CHEBI_16261> .'

In [55]:
#get number of each metaedge per subject and object metanode
##don't include labels
metaedgeinf = {}
for triple in mrinf[0:14489]:
    if triple == '':
        continue
    triple_list = triple.split('>')
    subj = triple_list[0]
    obj = triple_list[2]
    pred = triple_list[1]
    
    subj_id = subj.split('/')[-1].split('_')[0]
    pred_id = pred.split('/')[-1]
    if triple_list[1] == '<http://www.w3.org/2000/01/rdf-schema#label>':
        obj_id = ''
    else:
        obj_id = obj.split('/')[-1].split('_')[0]
    
    if obj_id != '':
        edgeinfo = subj_id + '-'+ pred_id + '-' + obj_id
    
    if edgeinfo not in metaedgeinf:
        metaedgeinf[edgeinfo] = 1
    else:
        metaedgeinf[edgeinfo] += 1
metaedgeinf

{'NCBITaxon-RO_0002434-NCBITaxon': 42,
 'UBERON-BFO_0000050-UBERON': 96,
 'UBERON-BFO_0000050-NCBITaxon': 251,
 'UBERON-BFO_0000050-CLO': 1,
 'CHEBI-RO_0002434-CHEBI': 1067,
 'CHEBI-RO_0002434-PR': 425,
 'CHEBI-RO_0002213-CHEBI': 3368,
 'CHEBI-RO_0002213-CL': 58,
 'CHEBI-RO_0002213-PR': 5283,
 'CHEBI-RO_0002213-napdi': 170,
 'GO-RO_0002434-PR': 25,
 'CHEBI-BFO_0000050-UBERON': 228,
 'GO-BFO_0000050-UBERON': 13,
 'PR-BFO_0000050-UBERON': 74,
 'napdi-BFO_0000050-UBERON': 3,
 'NCBITaxon-RO_0002434-CHEBI': 10,
 'CL-BFO_0000050-NCBITaxon': 162,
 'CL-BFO_0000050-UBERON': 84,
 'CHEBI-RO_0002213-GO': 252,
 'CHEBI-BFO_0000050-NCBITaxon': 272,
 'PR-BFO_0000050-NCBITaxon': 105,
 'CL-RO_0002434-CL': 16,
 'NCBITaxon-BFO_0000050-UBERON': 3,
 'PR-BFO_0000050-DOID': 1,
 'PR-BFO_0000050-CL': 2,
 'PR-BFO_0000050-SO': 1,
 'CHEBI-RO_0002434-MONDO': 6,
 'cellline#human-BFO_0000050-NCBITaxon': 25,
 'cellline#human-RO_0002434-PR': 1,
 'GO-RO_0002434-CL': 8,
 'GO-RO_0002434-CHEBI': 30,
 'CL-RO_0002434-PR': 29

In [59]:
for key in metaedgeinf:
    if metaedgeinf[key] >= 50:
        print(key, metaedgeinf[key])

UBERON-BFO_0000050-UBERON 96
UBERON-BFO_0000050-NCBITaxon 251
CHEBI-RO_0002434-CHEBI 1067
CHEBI-RO_0002434-PR 425
CHEBI-RO_0002213-CHEBI 3368
CHEBI-RO_0002213-CL 58
CHEBI-RO_0002213-PR 5283
CHEBI-RO_0002213-napdi 170
CHEBI-BFO_0000050-UBERON 228
PR-BFO_0000050-UBERON 74
CL-BFO_0000050-NCBITaxon 162
CL-BFO_0000050-UBERON 84
CHEBI-RO_0002213-GO 252
CHEBI-BFO_0000050-NCBITaxon 272
PR-BFO_0000050-NCBITaxon 105
PR-RO_0002434-CHEBI 51
PR-RO_0002434-PR 53
PR-RO_0002213-CHEBI 168
PR-RO_0002213-PR 371
CHEBI-RO_0002434-GO 68
napdi-RO_0002213-PR 137
napdi-RO_0002213-CHEBI 126
GO-RO_0002213-PR 100
napdi-RO_0002434-CHEBI 122
CHEBI-RO_0002434-napdi 123
napdi-RO_0002434-PR 97
SO-RO_0002213-CHEBI 70
CHEBI-RO_0002213-SO 98
SO-RO_0002213-PR 121
GO-RO_0002213-CHEBI 57


In [58]:
relations = {}
for triple in mrinf[0:14489]:
    if triple == '':
        continue
    triple_list = triple.split('>')

    pred = triple_list[1]
    pred_id = pred.split('/')[-1]
    
    if pred_id not in relations:
        relations[pred_id] = 1
    else:
        relations[pred_id] += 1
relations

{'RO_0002434': 2381, 'BFO_0000050': 1485, 'RO_0002213': 10623}

In [2]:
##Scratch

In [56]:
sem = pd.read_csv('semrep_data/semrep_all_predications_mapped_with_prefix_20220505.csv')
sem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145638 entries, 0 to 145637
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   pmid           145638 non-null  int64  
 1   subject_cui    141153 non-null  object 
 2   subject_type   145638 non-null  object 
 3   object_cui     141410 non-null  object 
 4   object_type    145638 non-null  object 
 5   year           145492 non-null  float64
 6   subject_name   141153 non-null  object 
 7   object_name    141410 non-null  object 
 8   source_text    145638 non-null  object 
 9   predicate      145638 non-null  object 
 10  subject_map    0 non-null       float64
 11  object_map     0 non-null       float64
 12  predicate_obo  145638 non-null  object 
 13  subject_obo    103194 non-null  object 
 14  object_obo     75713 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 16.7+ MB


In [45]:
reach = pd.read_csv('predication_files/kratom_pmid_all_predicates_umls_processed.tsv', sep='\t')
reach.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_cui           322 non-null    object 
 1   subject_name          322 non-null    object 
 2   subject_source        334 non-null    object 
 3   predicate             334 non-null    object 
 4   object_source         334 non-null    object 
 5   object_cui            327 non-null    object 
 6   object_name           327 non-null    object 
 7   subj_reach_grounding  334 non-null    object 
 8   obj_reach_grounding   334 non-null    object 
 9   pmid                  334 non-null    int64  
 10  year                  334 non-null    object 
 11  belief                334 non-null    float64
 12  sentence              334 non-null    object 
 13  predicate_obo         334 non-null    object 
 14  subject_obo           334 non-null    object 
 15  object_obo            3

In [63]:
x = sem.loc[sem['subject_name'] == 'mitragynine']['object_name'].tolist()

In [64]:
set(x)

{'4-methylumbelliferyl glucuronide',
 'ABCG2 gene',
 'Adenosine Triphosphatases',
 'Analgesics',
 'Atenolol',
 'Avoidance Learning',
 'Azithromycin',
 'Blockade',
 'Buffers',
 'CNS disorder',
 'CYP1A2 gene',
 'CYP2C9 protein, human',
 'CYP2D6 gene',
 'Calcium ion',
 'Cause of death.underlying',
 'Choroid Plexus Papilloma',
 'Cytochrome P450',
 'Cytochrome P450 2D6, human',
 'Dextromethorphan',
 'Did not receive therapy or drug for',
 'Digoxin',
 'Dimethyl Sulfoxide',
 'Dinoprostone',
 'Enzymatic Activity',
 'Enzymes, hematological',
 'Genus Homo',
 'Glutathione',
 'Glycine dehydrogenase (decarboxylating)',
 'Green Fluorescent Proteins',
 'Herb-Drug Interactions',
 'Homo sapiens',
 'Inflammation',
 'Insomnia, CTCAE 3.0',
 'KCNH2 gene',
 'KCNJ2 gene',
 'Kinetics',
 'Locomotion',
 'Loss of appetite (finding)',
 'Metabolism',
 'Methanol',
 'Midazolam',
 'Nerve',
 'Nicotine',
 'Norepinephrine',
 'Opioid Receptor',
 'P-Glycoproteins',
 'PHOSPHATE BUFFERS',
 'Palmitic Acid',
 'Potassium Chann

In [69]:
sem.loc[(sem['subject_obo'] == 'http://purl.obolibrary.org/obo/CHEBI_180536')]['source_text'][139612]

'26692748_ascii.txt.tx.4 However,  the  IC50  values  for the mitragynine and 7-hydroxymitragynine could not be determined  accurately  since  their  inhibitions  on  4-MU glucuronidation were <50%.4-MU glucuronidation was reduced by 7-hydroxymitragynine ranged from 6% +/- 2% to 46% +/- 1% and mitragynine ranged from 14% +/- 4% to 40% +/- 5% at concentrations of 0.01-100 uM [Figure 2].'